# Add targets for camera calibration

In [ ]:
import os
from glob import glob
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr

inputs_folder = '/Users/rdcrlrka/Research/Soo_locks/inputs'
cam_positions_file = os.path.join(inputs_folder, 'cams.txt')
ortho_files = sorted(glob(os.path.join(inputs_folder, '..', 'outputs', 'soo_locks_photogrammetry_20251001171000', 'orthoimages', '*.tiff')))
refdem_file = os.path.join(inputs_folder, 'lidar_DSM_filled_cropped.tif')
refl_file = os.path.join(inputs_folder, '20251001_Soo_Model_1cm_Intensity_UTM19N-fake.tif')

output_folder = os.path.join(inputs_folder, '..', 'add_targets')
os.makedirs(output_folder, exist_ok=True)

# Load camera positions
cams = pd.read_csv(cam_positions_file, sep=' ', header=0)
cams['channel'] = [x.split('_')[1] for x in cams['img_name']]

# imoprt utility functions
import model_camera_coverage_utils as utils

# Load current target locations
targets_file = os.path.join(inputs_folder, '..', 'add_targets', 'current_targets.gpkg')
targets = gpd.read_file(targets_file)
# grab x and y coordinates
targets['X'] = [x.coords.xy[0][0] for x in targets['geometry']]
targets['Y'] = [x.coords.xy[1][0] for x in targets['geometry']]
# sort by y-value
targets = targets.sort_values(by='Y').reset_index(drop=True)

# Load lidar DSM
dem = rxr.open_rasterio(refdem_file).squeeze()
dem = xr.where(dem==-9999, np.nan, dem)
# upsample to speed up plotting
new_x = np.arange(dem.x.min().data, dem.x.max().data, step=0.05)
new_y = np.arange(dem.y.min().data, dem.y.max().data, step=0.05)
dem = dem.interp(coords={'x': new_x, 'y': new_y}, method='linear')

## Calculate current image bounds

In [ ]:
print('Creating polygon of model space')
model_space_gdf = utils.calculate_image_footprint(refdem_file)

print('Calculating image footprints')
gdf_list = []
for ortho_file in tqdm(ortho_files):
    gdf = utils.calculate_image_footprint(ortho_file)
    gdf_list += [gdf]
bounds_df = pd.concat(gdf_list).reset_index(drop=True)
bounds_gdf = gpd.GeoDataFrame(bounds_df, geometry=bounds_df['geometry'], crs=gdf.crs)

print('Calculating number of targets in each image')
def count_image_targets(targets_gdf, image_bounds_gdf):
    df_list = []
    for i in range(len(image_bounds_gdf)):
        im_gdf = image_bounds_gdf.iloc[i]
        intersects = [im_gdf['geometry'].intersects(x) for x in targets_gdf['geometry']]
        num_intersecting = np.sum(np.array(intersects))
        df_list += [pd.DataFrame({
            'channel': [im_gdf['channel']],
            'num_targets': [num_intersecting]
        }, index=[i])]
    df_full = pd.concat(df_list)
    return df_full

image_targets_df = count_image_targets(targets, bounds_gdf)
image_targets_df

## Add targets

In [ ]:
new_target_pts = []

def add_points(x1, y1, rot, dy, npoints):
    # new points
    xnew = x1 * np.ones(npoints)
    ynew = y1 + dy*(1+np.arange(npoints))
    series = gpd.GeoSeries([LineString([(x1,y1)] + list(zip(xnew,ynew)))])
    # rotate to align with sides
    series_rot = series.rotate(rot, origin=(x1,y1))
    new_x, new_y = series_rot[0].coords.xy
    return [Point(x,y) for (x,y) in list(zip(new_x, new_y))]

dy = 1.5

### SE ###
x1, y1 = (0.6, 5)
rot = 16
if dy==2:
    npoints = 14
elif dy==1.5:
    npoints = 19
new_target_pts += add_points(x1, y1, rot, dy, npoints)

### SW ###
x1, y1 = (-6.5, 3)
rot = 14
if dy==2:
    npoints = 11
elif dy==1.5:
    npoints = 15
new_target_pts += add_points(x1, y1, rot, dy, npoints)

### Walkway ###
x1, y1 = (-2.8, 4)
rot = 15
if dy==2:
    npoints = 18
elif dy==1.5:
    npoints = 24
new_target_pts += add_points(x1, y1, rot, dy, npoints)

### NE ###
x1, y1 = (-7.4, 34.2)
rot = 14.5
if dy==2:
    npoints = 13
elif dy==1.5:
    npoints = 18
new_target_pts += add_points(x1, y1, rot, dy, npoints)

### NW1 ###
x1, y1 = (-12.5, 26.5)
rot = 17
if dy==2:
    npoints = 12
elif dy==1.5:
    npoints = 17
new_target_pts += add_points(x1, y1, rot, dy, npoints)

### NW2 ###
x1, y1 = (-20.5, 52)
rot = 28
if dy==2:
    npoints = 3
elif dy==1.5:
    npoints = 5
new_target_pts += add_points(x1, y1, rot, dy, npoints)
new_target_pts += [Point(-25.5, 59.2)]

# Create geodataframe
new_targets = gpd.GeoDataFrame(geometry=new_target_pts, crs="EPSG:32619")

# Calculate new number of targets per image
new_image_targets_df = count_image_targets(new_targets, bounds_gdf)

# Save to file
out_file = os.path.join(output_folder, f'add_{len(new_targets)}_targets_{dy}m.csv')
new_image_targets_df.to_csv(out_file, index=False)
print(f'Targets per image saved to:\n{out_file}')

print('\nNumer of targets per image:')
print('\tMean = ', new_image_targets_df['num_targets'].mean())
print('\tMin = ', new_image_targets_df['num_targets'].min())
print('\tMax = ', new_image_targets_df['num_targets'].max())
new_image_targets_df


## Plot results

In [ ]:
plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Verdana'})
fig, ax = plt.subplots(figsize=(8,14))

# model space
model_space_gdf.plot(facecolor='None', edgecolor='k', ax=ax)

# lidar DSM
(xr.where(dem > -8, np.nan, dem)).plot(cmap='Grays', vmin=-8.2, vmax=-7.9, add_colorbar=False)

# new targets
new_targets.plot(ax=ax, facecolor='m', edgecolor='w', linewidth=0.5, markersize=50)

ax.set_title(f'Add {len(new_targets)} targets at {dy} m spacing')
ax.set_xlabel('meters')
ax.set_ylabel('meters')
fig.tight_layout()
plt.show()

# Save figure
fig_file = os.path.join(output_folder, f'add_{len(new_targets)}_targets_{dy}m.png')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print(f"Figure saved to:\n{fig_file}")